In [5]:
# predict which theme ...
import pandas as pd
import numpy as np

In [6]:
# read data -- sets
setsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/sets.csv'
sets = pd.read_csv(setsfile)
#sets.head()

In [7]:
themefile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/themes.csv'
themes = pd.read_csv(themefile)
#themes.head()

In [8]:
setsinvenfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventory_sets.csv'
invenotry_sets = pd.read_csv(setsinvenfile)

In [9]:
invenfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventories.csv'
invenotries = pd.read_csv(invenfile)
#invenotries.head()

In [10]:
invenpartsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventory_parts.csv'
invenotry_parts = pd.read_csv(invenpartsfile)

In [11]:
partsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/parts.csv'
parts = pd.read_csv(partsfile)
#parts.head()

In [12]:
#join sets and themes together
t1 = pd.merge(sets,themes, left_on = 'theme_id', right_on = 'id', suffixes = ('_set','_theme'))

In [13]:
# join t1 with ivnetories, inventoryset inventory parts
t4 = pd.merge(t1,invenotry_sets, on = 'set_num')
t4.head()

,set_num,name_set,year,theme_id,num_parts,id,name_theme,parent_id,inventory_id,quantity
0,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1
1,6041-1,Armor Shop,1986,199,113,199,Lion Knights,186.0,2216,1
2,10000-1,Guarded Inn,2001,186,256,186,Castle,NaN,7474,1
3,10039-1,Black Falcon's Fortress,2002,186,431,186,Castle,NaN,7474,1
4,1794-1,Dragon Master Chariot,1994,186,36,186,Castle,NaN,5097,1


In [14]:
# then join parts and color and categories, and then join sets with them

#join t2 with color (t2 join inventory and inventory join color)
t3 = pd.merge(parts, invenotry_parts, on = 'part_num')
# drop useless columns
t3.head()

,part_num,name,part_cat_id,inventory_id,color_id,quantity,is_spare
0,0687b1,Set 0687 Activity Booklet 1,17,15235,9999,1,f
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,1973,2,1,f
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,1973,2,1,f
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1,1973,2,1,f
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1,1973,2,1,f


In [15]:
t5 = pd.merge(t4, t3, left_on = 'id', right_on = 'inventory_id', how = 'inner',suffixes = ('_inven','_parts'))
t5.head()

,set_num,name_set,year,theme_id,num_parts,id,name_theme,parent_id,inventory_id_inven,quantity_inven,part_num,name,part_cat_id,inventory_id_parts,color_id,quantity_parts,is_spare
0,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,10197,Technic Pin Connector Hub with 2 Perpendicular...,12,199,72,2,f
1,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,11334,Hero Factory Armour with Spikes,41,199,15,2,f
2,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15365pat0001,Bionicle Zamor Sphere with Marbled White / Blu...,41,199,1,1,f
3,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15366,Hero Factory Full Torso Armor with Connector f...,41,199,148,1,f
4,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15376pat01,Large Figure Head Modified Chima Mammoth with ...,41,199,70,1,f


# how have the size of set changed over time?



In [12]:

## select part_num, nset_num, setname and year 
partdf = t5.filter(['part_num','set_num', 'year'], axis=1)
part1 = partdf.groupby(["year", "set_num"]).size().reset_index(name="num_of_parts")
print(part1.head())


   year set_num  num_of_parts
0  1970   603-3             6
1  1980  1062-1             1
2  1981  1064-1             1
3  1981   322-1             6
4  1981  6611-1           138


In [13]:
# count the size of parts based on year
part2 = part1.drop(part1.columns[[1]], axis=1)
part2.groupby('year')['num_of_parts'].sum()

# draw plot
import matplotlib.pyplot as plt
x = part2['year']
y = part2['num_of_parts']
plt.bar(x,y)

#set to index 
part3 = part2.set_index('year')
part3.plot()

# Could you predict which theme a set is from just by the bricks it contains? 

In [14]:
# using size of parts in set to predict which theme is belongs
df_new = t5.filter(['name_part','name_theme'], axis=1)
df_q3 = df_new.groupby(["name_theme"]).size().reset_index(name="num_of_parts")

print(df_q3.head())

       name_theme  num_of_parts
0              9V           168
1  Advent Sub-Set          2640
2         Airport           423
3         Animals            33
4  Aquaraiders II            13


In [15]:
# divide training set, validation set and testing set
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_q3, test_size=0.4)  #60% training test and 40% testing 

In [16]:
# check how many rows in the training dataset
print(len(train.index))

89


In [17]:
# check how many rows in the testing dataset
print(len(test.index))

60


In [18]:
train.head()

,name_theme,num_of_parts
137,Toa Nuva,760
121,Star Wars Episode 4/5/6,306
106,Series 2 Minifigures,85
57,Hockey,58
3,Animals,33


In [19]:
list(train) 

['name_theme', 'num_of_parts']

In [20]:
# explanotary variables: both categorical and numeric
# response variable: categorical 


In [21]:
# unique theme
len(df_q3['name_theme'].unique())


149

In [22]:
train.isnull().sum() #no null value in train dataset

name_theme      0
num_of_parts    0
dtype: int64

In [35]:
# using naive bayes to predict
type(train)
train['name_theme'].unique()

array(['Toa Nuva', 'Star Wars Episode 4/5/6', 'Series 2 Minifigures',
       'Hockey', 'Animals', 'Visorak', 'Gravity Games', 'Castle',
       'Series 15 Minifigures', 'Station', 'Toa Hagah', 'DFB Minifigures',
       'Lion Knights', 'Divers', 'Designer Sets', 'Creature', 'Barraki',
       'Creator', 'Police', 'Train', 'Traffic', 'Hospital',
       'Universal Building Set', 'Spyrius', 'Coast Guard', 'Pirates II',
       'Batman', 'Fire', 'Battle Vehicles', 'Basic', 'Belville',
       'Prisoner of Azkaban', 'Planet Series 2', 'Advent Sub-Set',
       'Holiday', 'Technic', 'Matoran of Mata Nui', 'Wolfpack',
       'Bulk Bricks', 'Power Racers', 'Knights Kingdom II', 'Sculptures',
       'Matoran of Light', 'Speed Champions', 'Cars', 'Rock Raiders',
       'Turaga', 'Toa Hordika', 'Mars Mission', 'Pirates', 'Heroes',
       'Mistika', 'Robot', 'Fantasy Era', 'Launch Command', '9V',
       'Spider-Man', 'Matoran of Metru Nui', 'Discovery', 'Arctic',
       'Team GB', 'Forestmen', 'Legends 

In [52]:
#Import Library of Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [45]:
model = GaussianNB()

In [1]:
x = train['num_of_parts'].values.reshape(1,-1) 
y = train['name_theme']

model.fit(x, y)

NameError: name 'train' is not defined

# What sets have the rarest pieces in them? 

In [17]:
# what is the rarest pieces in each sets?
# group by set, show part names
t5.head()


,set_num,name_set,year,theme_id,num_parts,id,name_theme,parent_id,inventory_id_inven,quantity_inven,part_num,name,part_cat_id,inventory_id_parts,color_id,quantity_parts,is_spare
0,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,10197,Technic Pin Connector Hub with 2 Perpendicular...,12,199,72,2,f
1,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,11334,Hero Factory Armour with Spikes,41,199,15,2,f
2,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15365pat0001,Bionicle Zamor Sphere with Marbled White / Blu...,41,199,1,1,f
3,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15366,Hero Factory Full Torso Armor with Connector f...,41,199,148,1,f
4,6016-1,Knights' Arsenal,1987,199,37,199,Lion Knights,186.0,2216,1,15376pat01,Large Figure Head Modified Chima Mammoth with ...,41,199,70,1,f


In [27]:
q4 = t5[['name_set','name']]


In [44]:
dfnew = q4.groupby(by=['name_set'])

In [50]:
data = q4.groupby(['name_set', 'name']).size()

In [59]:
data.head()

name_set                   name                                        
 Spectre                   Technic Digger Bucket 4 x 4 x 9 with 3 teeth    1
                           Technic Digger Bucket 8 x 10                    1
                           Technic Metal Hook - Large                      1
'Where Are My Pants?' Guy  Antenna 1 x 4 with Rounded Top                  1
                           Baseplate 8 x 16                                1
dtype: int64

In [60]:
type(data)

pandas.core.series.Series

In [74]:
# if all parts number is 1, there is no rarest parts
# if the parts number change a lot, and has obvious small sparts numbers
d1 = dict(data)

In [71]:
type(d1)


builtin_function_or_method